In [ ]:
import glob
import sys
import json

In [ ]:
submission_dir='submissions/'
gold_dir='ref/'
input_dir='input/'

answers_file='answers.json'
questions_file='questions.json'

subtasks=['s1','s2','s3']

In [ ]:
# Preload questions
questions={}
for subtask in subtasks:
    subtask_questions_file='%s/%s/%s' % (input_dir, subtask, questions_file)
    with open(subtask_questions_file, 'r') as f:
        questions[subtask]=json.load(f)

In [ ]:
gold={}
for subtask in subtasks:
    subtask_gold_file='%s/%s/%s' % (gold_dir, subtask, answers_file)
    with open(subtask_gold_file, 'r') as f:
        gold[subtask]=json.load(f)

In [ ]:
def compute_subset_accuracy(ques, anss, gold, event_type=None, event_prop=None):
    correct=0
    total=0
    for qid, adata in anss.items():
        if event_type and event_type!=ques[qid]['event_type']:
            continue
        if event_prop and event_prop not in ques[qid].keys():
            continue
        num_answer=adata['numerical_answer']
        if num_answer==gold[qid]['numerical_answer']:
            correct+=1
        total+=1
    print(correct, total)
    return correct/total

In [ ]:
for user_submission_dir in glob.glob('%s/*' % submission_dir):
    print(user_submission_dir)
    for subtask_user_submission_dir in glob.glob('%s/*' % user_submission_dir):
        subtask=subtask_user_submission_dir.split('/')[-1]
        print(subtask)
        this_answers_file='%s/%s' % (subtask_user_submission_dir, answers_file)
        with open(this_answers_file, 'r') as f:
            answers = json.load(f)
            acc=compute_subset_accuracy(questions[subtask], 
                                    answers, 
                                    gold[subtask], 
                                    event_type='killing')
            print(acc)
